## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-07-13-07-01 +0000,nypost,Warner Bros. Discovery rejects latest takeover...,https://nypost.com/2026/01/07/business/warner-...
1,2026-01-07-13-05-00 +0000,wsj,U.S. Forces Launch Operation to Seize Fleeing ...,https://www.wsj.com/business/logistics/u-s-for...
2,2026-01-07-13-03-52 +0000,nyt,Live Updates: Trump Says Venezuela Will Send O...,https://www.nytimes.com/live/2026/01/07/world/...
3,2026-01-07-12-58-00 +0000,wsj,Albertsons Sales Rise on Pharmacy and Digital ...,https://www.wsj.com/business/retail/albertsons...
4,2026-01-07-12-57-49 +0000,nyt,Buy Greenland? Take It? Why? An Old Pact Alrea...,https://www.nytimes.com/2026/01/07/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,53
21,venezuela,49
16,oil,19
297,maduro,18
99,new,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
218,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...,164
73,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,155
160,2026-01-07-00-32-44 +0000,nypost,Venezuela selling up to 50 million barrels of ...,https://nypost.com/2026/01/06/us-news/venezuel...,146
94,2026-01-07-09-07-00 +0000,wsj,Venezuela to Give U.S. Up to 50 Million Barrel...,https://www.wsj.com/politics/policy/venezuela-...,145
131,2026-01-07-03-19-29 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,143


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
218,164,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...
73,102,2026-01-07-10-10-24 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...
85,65,2026-01-07-10-00-00 +0000,wsj,The welfare-fraud scandal that led Minnesota G...,https://www.wsj.com/politics/elections/minneso...
290,55,2026-01-06-17-47-55 +0000,nypost,AI deepfakes of Nicolás Maduro flood social me...,https://nypost.com/2026/01/06/business/ai-deep...
233,54,2026-01-06-21-51-00 +0000,wsj,Secretary of State Rubio told lawmakers that r...,https://www.wsj.com/politics/national-security...
188,45,2026-01-06-23-17-54 +0000,latimes,New California tool can stop brokers from sell...,https://www.latimes.com/california/story/2026-...
50,43,2026-01-07-11-00-00 +0000,nypost,LA fire victims have just bills and broken pro...,https://nypost.com/2026/01/07/us-news/la-fire-...
40,37,2026-01-07-11-42-21 +0000,nypost,‘Squad’ Rep. Ayanna Pressley’s ex-con husband ...,https://nypost.com/2026/01/07/us-news/ayanna-p...
311,31,2026-01-06-16-16-08 +0000,nypost,Security clash with protesters in Iran’s Grand...,https://nypost.com/2026/01/06/world-news/secur...
76,31,2026-01-07-10-04-09 +0000,nypost,South Korean leader says he asked China’s Xi t...,https://nypost.com/2026/01/07/world-news/south...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
